In [1]:
%reload_ext autoreload
%autoreload 2

## Setup

In [2]:
from math import isclose
from ipyexperiments import IPyExperimentsCPU
import ipyexperiments
import re
from utils.text import *

In [3]:
"""test_version"""
assert ipyexperiments.__version__, "version check"

'test_version'

## Basic container test


In [4]:
%%capture output
locals_unset(['exp1', 'x1', 'x2'])

exp1 = IPyExperimentsCPU(cl_enable=False)

x2 = consume_cpu_ram_128mb()
x1 = consume_cpu_ram_128mb()
check_defined(['x1', 'x2'])

#exp1.finish()
del exp1
check_undefined(['x1', 'x2'])

# cleanup
locals_unset(['exp1', 'x1', 'x2'])

In [5]:
output = str(output)
print_output(output)
check_reclaimed(output)

Captured output:
| 
| *** Experiment started with the CPU-only backend
| 
| 
| *** Current state:
| RAM:  Used  Free  Total      Util
| CPU:   163  7288  31588 MB   0.52% 
| 
| 
| 
| IPyExperimentsCPU: Finishing
| 
| *** Experiment finished in 00:00:00 (elapsed wallclock time)
| 
| *** Newly defined local variables:
| Deleted: x1, x2
| 
| *** Experiment memory:
| RAM:  Consumed     Reclaimed
| CPU:     255     255 MB (100.00%)
| 
| *** Current state:
| RAM:  Used  Free  Total      Util
| CPU:   163  7285  31588 MB   0.52% 
| 
| 
| 



## Test stats data and selective vars preservation

In [6]:
%%capture output
exp2 = IPyExperimentsCPU(cl_enable=False) 
x1 = consume_cpu_ram_128mb()

cpu_data = exp2.data
assert cpu_data
print(cpu_data)

exp2.keep_var_names('cpu_data')

x2 = consume_cpu_ram_128mb() 

cpu_data = exp2.data
print(cpu_data)

check_defined(['x1', 'x2', 'cpu_data'])
    
cpu_data_final = exp2.finish() # finish experiment
print("\nNumerical data:\n", cpu_data_final)

check_defined(['cpu_data_final'])
check_undefined(['x1', 'x2'])

In [7]:
output = str(output)
print_output(output)
check_reclaimed(output)
check_data(output, cpu_data_final)

Captured output:
| 
| *** Experiment started with the CPU-only backend
| 
| 
| *** Current state:
| RAM:  Used  Free  Total      Util
| CPU:   163  7283  31588 MB   0.52% 
| 
| 
| IPyExperimentMemory(consumed=134074368, reclaimed=0, available=7502958592)
| IPyExperimentMemory(consumed=268431360, reclaimed=0, available=7366856704)
| 
| IPyExperimentsCPU: Finishing
| 
| *** Experiment finished in 00:00:00 (elapsed wallclock time)
| 
| *** Newly defined local variables:
| Deleted: x1, x2
| Kept:    cpu_data
| 
| *** Experiment memory:
| RAM:  Consumed     Reclaimed
| CPU:     255     255 MB ( 99.97%)
| 
| *** Current state:
| RAM:  Used  Free  Total      Util
| CPU:   163  7281  31588 MB   0.52% 
| 
| 
| 
| Numerical data:
|  IPyExperimentMemory(consumed=268431360, reclaimed=268361728, available=7634755584)
| 



## Using the context manager

If you want to put all cells into one, you could simplify the experiment even further by using its context manager.

In [8]:
%%capture output
with IPyExperimentsCPU(cl_enable=False):
    x1 = consume_cpu_ram_128mb()
    x2 = consume_cpu_ram_128mb()
    
check_undefined(['x1', 'x2'])

In [9]:
output = str(output)
print_output(output)
check_reclaimed(output)

Captured output:
| 
| *** Experiment started with the CPU-only backend
| 
| 
| *** Current state:
| RAM:  Used  Free  Total      Util
| CPU:   163  7270  31588 MB   0.52% 
| 
| 
| 
| IPyExperimentsCPU: Finishing
| 
| *** Experiment finished in 00:00:00 (elapsed wallclock time)
| 
| *** Newly defined local variables:
| Deleted: x1, x2
| 
| *** Experiment memory:
| RAM:  Consumed     Reclaimed
| CPU:     256     255 MB ( 99.99%)
| 
| *** Current state:
| RAM:  Used  Free  Total      Util
| CPU:   163  7267  31588 MB   0.52% 
| 
| 
| 



In [10]:
%%capture output
with IPyExperimentsCPU(cl_enable=False) as exp: 
    x1 = consume_cpu_ram_128mb()
    z = "some data"
    x2 = consume_cpu_ram_128mb()
    exp.keep_var_names('z')
print(z)
check_defined(['z'])
check_undefined(['x1', 'x2'])

In [11]:
output = str(output)
print_output(output)
check_reclaimed(output)

Captured output:
| 
| *** Experiment started with the CPU-only backend
| 
| 
| *** Current state:
| RAM:  Used  Free  Total      Util
| CPU:   163  7264  31588 MB   0.52% 
| 
| 
| 
| IPyExperimentsCPU: Finishing
| 
| *** Experiment finished in 00:00:00 (elapsed wallclock time)
| 
| *** Newly defined local variables:
| Deleted: x1, x2
| Kept:    z
| 
| *** Experiment memory:
| RAM:  Consumed     Reclaimed
| CPU:     256     255 MB ( 99.99%)
| 
| *** Current state:
| RAM:  Used  Free  Total      Util
| CPU:   163  7260  31588 MB   0.52% 
| 
| 
| some data
| 



## Deleting previously defined variables

Here we test that we **cannot** correctly detect and delete variables used in the scope of the experiment, but which were already defined prior to the experiment.

In [12]:
%%capture output
x3 = 10
x4 = 20
with IPyExperimentsCPU(cl_enable=False): 
    x1 = consume_cpu_ram_128mb()
    x2 = consume_cpu_ram_128mb()
    x3 = 10
    x4 = 10000
# x3 is the same value, so we can't tell the difference whether it was created before the experiment or not
# x4 is different from previous value, but it could have been modified by some function indirectly and not used directly in the experiment
check_defined(['x3', 'x4'])
# x3 is newly defined, so it gets deleted
check_undefined(['x1', 'x2'])

In [13]:
output = str(output)
print_output(output)
check_reclaimed(output)

Captured output:
| 
| *** Experiment started with the CPU-only backend
| 
| 
| *** Current state:
| RAM:  Used  Free  Total      Util
| CPU:   163  7260  31588 MB   0.52% 
| 
| 
| 
| IPyExperimentsCPU: Finishing
| 
| *** Experiment finished in 00:00:00 (elapsed wallclock time)
| 
| *** Newly defined local variables:
| Deleted: x1, x2
| 
| *** Experiment memory:
| RAM:  Consumed     Reclaimed
| CPU:     256     255 MB ( 99.99%)
| 
| *** Current state:
| RAM:  Used  Free  Total      Util
| CPU:   163  7254  31588 MB   0.52% 
| 
| 
| 



Same but with the CellLogger enabled

In [14]:
%%capture output
x3 = 10
x4 = 20
with IPyExperimentsCPU(cl_enable=True): 
    x1 = consume_cpu_ram_128mb()
    x2 = consume_cpu_ram_128mb()
    x3 = 10
    x4 = 10000
# x3 is the same value, so we can't tell the difference whether it was created before the experiment or not
# x4 is different from previous value, but it could have been modified by some function indirectly and not used directly in the experiment
check_defined(['x3', 'x4'])
# x3 is newly defined, so it gets deleted
check_undefined(['x1', 'x2'])

In [15]:
output = str(output)
print_output(output)
check_reclaimed(output)

Captured output:
| 
| *** Experiment started with the CPU-only backend
| 
| 
| *** Current state:
| RAM:  Used  Free  Total      Util
| CPU:   163  7249  31588 MB   0.52% 
| 
| 
| ･ RAM: △Consumed △Peaked  Used Total | Exec time 0.059s
| ･ CPU:       256       0      419 MB |
| 
| IPyExperimentsCPU: Finishing
| 
| *** Experiment finished in 00:00:00 (elapsed wallclock time)
| 
| *** Newly defined local variables:
| Deleted: x1, x2
| 
| *** Experiment memory:
| RAM:  Consumed     Reclaimed
| CPU:     256     255 MB ( 99.99%)
| 
| *** Current state:
| RAM:  Used  Free  Total      Util
| CPU:   163  7214  31588 MB   0.52% 
| 
| 
| 



In [16]:
%%javascript # prevent committing an unsaved notebook
IPython.notebook.save_notebook()

<IPython.core.display.Javascript object>